In [1]:
pip install pandas requests gradio python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
import pandas as pd
import requests
import gradio as gr
from typing import List, Dict, Optional

# --- Configuration --- #
TMDB_API_KEY = os.getenv('TMDB_API_KEY', 'd25484ce9c93bdb1747d679edaa10f8c')

# --- Data Loading & Processing --- #
def load_data():
    """Load and preprocess movie data."""
    try:
        movies = pd.read_csv('/content/movie.csv')
        ratings = pd.read_csv('/content/rating.csv')
        genome_tags = pd.read_csv('/content/genome_tags.csv')
        genome_scores = pd.read_csv('/content/genome_scores.csv')

        genome_df = pd.merge(genome_scores, genome_tags, on='tagId')

        top_tags = (
            genome_df.sort_values(['movieId', 'relevance'], ascending=[True, False])
            .groupby('movieId').head(10)
            .groupby('movieId')['tag'].apply(list).reset_index()
        )

        movies_with_tags = pd.merge(movies, top_tags, on='movieId', how='left')
        avg_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
        movies_with_ratings = pd.merge(movies_with_tags, avg_ratings, on='movieId')

        movies_with_ratings['year'] = movies_with_ratings['title'].str.extract(r'\((\d{4})\)')
        movies_with_ratings['tag'] = movies_with_ratings['tag'].apply(lambda x: x if isinstance(x, list) else [])

        return movies_with_ratings

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise

# --- TMDB API Integration --- #
def get_movie_details(title: str) -> Dict:
    """Fetch movie details from TMDB API with error handling."""
    try:
        clean_title = title.split('(')[0].strip()

        response = requests.get(
            "https://api.themoviedb.org/3/search/movie",
            params={
                "api_key": TMDB_API_KEY,
                "query": clean_title,
                "include_adult": "false"
            },
            timeout=10
        )
        response.raise_for_status()
        data = response.json()

        if data["results"]:
            movie = data["results"][0]
            return {
                "title": movie.get("title", title),
                "overview": movie.get("overview", "No overview available."),
                "rating": movie.get("vote_average", "N/A"),
                "poster_url": f"https://image.tmdb.org/t/p/w300{movie['poster_path']}" if movie.get("poster_path") else None,
                "release_date": movie.get("release_date", "Unknown")
            }
    except Exception as e:
        print(f"Error fetching data for {title}: {str(e)}")

    return {
        "title": title,
        "overview": "No TMDB data found.",
        "rating": "N/A",
        "poster_url": None,
        "release_date": "Unknown"
    }

# --- Recommendation Engine --- #
class MovieRecommender:
    def __init__(self, movie_data: pd.DataFrame):
        self.movies = movie_data
        self.mood_keywords = {
            "happy": ["comedy", "feel-good", "uplifting", "funny", "lighthearted"],
            "sad": ["drama", "emotional", "tragic", "heartbreaking", "melancholy"],
            "romantic": ["romance", "love", "relationship", "passion", "heartwarming"],
            "excited": ["action", "thriller", "adventure", "fast-paced", "intense"],
            "chill": ["calm", "relaxing", "slow", "meditative", "peaceful"],
            "scared": ["horror", "suspense", "terror", "creepy", "psychological"],
            "curious": ["mystery", "mind-bending", "sci-fi", "detective", "twist"]
        }

    def recommend(self, mood: str, top_n: int = 5) -> List[Dict]:
        """Recommend movies based on mood."""
        keywords = self.mood_keywords.get(mood.lower())
        if not keywords:
            return []

        df = self.movies.copy()
        df['match_score'] = df['tag'].apply(
            lambda tags: sum(1 for tag in tags for kw in keywords if kw in tag.lower())
        )

        recommendations = (
            df[df['match_score'] > 0]
            .sort_values(['match_score', 'rating'], ascending=[False, False])
            .head(top_n)
        )

        results = []
        for _, row in recommendations.iterrows():
            movie_info = get_movie_details(row['title'])
            enhanced_info = {
                **movie_info,
                "genres": row['genres'].replace('|', ', '),
                "year": row['year'],
                "avg_rating": round(row['rating'], 1)
            }
            results.append(enhanced_info)

        return results

# --- Gradio Interface --- #
def create_interface(recommender: MovieRecommender):
    """Build the Gradio UI with improved display."""
    with gr.Blocks(theme=gr.themes.Soft(), css=".movie-card {border-radius: 10px; padding: 15px; margin-bottom: 15px; background: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);}") as app:
        gr.Markdown("# 🎬 Mood-Based Movie Recommender")

        with gr.Row():
            mood_input = gr.Dropdown(
                list(recommender.mood_keywords.keys()),
                label="How are you feeling today?",
                value="happy"
            )
            count_input = gr.Slider(3, 10, value=5, label="Number of recommendations")

        btn = gr.Button("Find Movies", variant="primary")

        # Custom HTML output for better display
        html_output = gr.HTML()
        gallery = gr.Gallery(
            label="Movie Posters",
            object_fit="contain",
            height="auto",
            columns=3
        )

        def recommend_and_display(mood: str, n: int):
            movies = recommender.recommend(mood, n)

            # Build HTML cards for each movie
            html_cards = []
            for movie in movies:
                card = f"""
                <div class="movie-card">
                    <div style="display: flex; gap: 20px;">
                        {f'<img src="{movie["poster_url"]}" width="150" style="border-radius: 5px;">' if movie["poster_url"] else ''}
                        <div>
                            <h3>{movie["title"]} ({movie["year"]})</h3>
                            <p><strong>Genres:</strong> {movie["genres"]}</p>
                            <p><strong>Rating:</strong> ⭐ {movie["avg_rating"]}/5 (TMDB: {movie["rating"]}/10)</p>
                            <p><strong>Overview:</strong> {movie["overview"]}</p>
                        </div>
                    </div>
                </div>
                """
                html_cards.append(card)

            return (
                "<div style='display: flex; flex-direction: column; gap: 15px;'>" +
                "\n".join(html_cards) +
                "</div>",
                [m["poster_url"] for m in movies if m["poster_url"]]
            )

        btn.click(
            recommend_and_display,
            inputs=[mood_input, count_input],
            outputs=[html_output, gallery]
        )

    return app

# --- Main Execution --- #
if __name__ == "__main__":
    print("Loading data...")
    try:
        movie_data = load_data()
        recommender = MovieRecommender(movie_data)

        print("Launching app...")
        app = create_interface(recommender)
        app.launch(debug=True, share=True)
    except Exception as e:
        print(f"Failed to start application: {str(e)}")

Loading data...
Launching app...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://28b52bdf25ac1a90c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
